In [2]:
import scipy.stats
from sklearn.metrics import mean_squared_error
import pandas as pd
from matplotlib import pyplot as plt

In [28]:
regression_results = pd.read_csv("../results/multioutput/detection_results_all_feat.csv", index_col=0)
regression_results

,Matrix short,Polarity,regressor,observed_value,prediction,name_short,adduct
0,9AA,negative,Lin_reg,0,-2.135966e+10,2-Oxoglutaric acid,+Cl
1,9AA,positive,Lin_reg,0,-3.003984e+10,2-Oxoglutaric acid,+Cl
2,CHCA,negative,Lin_reg,0,-6.240325e+08,2-Oxoglutaric acid,+Cl
3,CHCA,positive,Lin_reg,0,-3.409644e+10,2-Oxoglutaric acid,+Cl
4,CMBT,negative,Lin_reg,0,-8.175534e+09,2-Oxoglutaric acid,+Cl
...,...,...,...,...,...,...,...
60055,NEDC,positive,GaussianProcessMultiOut,0,1.051754e-01,gamma-Aminobutyric acid,[M]-
60056,NOR,negative,GaussianProcessMultiOut,0,-1.869756e-01,gamma-Aminobutyric acid,[M]-
60057,NOR,positive,GaussianProcessMultiOut,0,3.206369e-01,gamma-Aminobutyric acid,[M]-
60058,pNA,negative,GaussianProcessMultiOut,0,-2.648387e-01,gamma-Aminobutyric acid,[M]-


In [19]:
# compute Spearman's/Pearson's correlation and mean squared error for each matrix/polarity/regressor
regression_metrics = pd.DataFrame(columns = ['matrix', 'polarity', 'regressor', "Spearman's R", 
                                             'S pval', "Pearson's R", 'P pval', 'RMSE', 'RMSE/std', 'non-zero obs'])
counter = 0
for (matrix, polarity, regressor), rows in regression_results.groupby(['Matrix short', 'Polarity', 'regressor']):
    # remove zero intensity molecules if needed (optional)
    rows = rows[rows['observed_value']!=0]
    
    spearman = scipy.stats.spearmanr(rows.observed_value, rows.prediction)
    pearson = scipy.stats.pearsonr(rows.observed_value, rows.prediction)
    mse = mean_squared_error(rows.observed_value, rows.prediction, squared = False)
    mse_std = mse / rows['observed_value'].std()
    regression_metrics.loc[counter] = [matrix, polarity, regressor, spearman[0], spearman[1], 
                                       pearson[0], pearson[1], mse, mse_std, 
                                       rows[rows['observed_value']!=0].shape[0]]
    counter += 1

In [20]:
# select best regressor for each matrix/polarity combination
best_RMSE = regression_metrics.loc[regression_metrics.groupby(['matrix', 'polarity'])["RMSE/std"].idxmin()]
#best_RMSE

best_spear = regression_metrics.loc[regression_metrics.groupby(['matrix', 'polarity'])["Spearman's R"].idxmax()].sort_values("Spearman's R", ascending=False)
best_spear

,matrix,polarity,regressor,Spearman's R,S pval,Pearson's R,P pval,RMSE,RMSE/std,non-zero obs
9,9AA,negative,RandomForest,0.745471,7.066228e-31,0.683869,2.312679e-24,1.681396,0.766526,167
22,9AA,positive,RandomForest,0.647281,1.492288e-13,0.684512,1.564091e-15,1.232718,0.756134,103
61,CMBT,negative,RandomForest,0.619202,2.826969e-07,0.441913,5.785716e-04,0.549919,0.985802,57
217,NOR,negative,RandomForest,0.618408,4.336882e-17,0.495378,1.337387e-10,1.462239,0.929936,149
230,NOR,positive,RandomForest,0.605522,2.211668e-17,0.624746,1.061220e-18,1.196835,0.823233,160
196,NEDC,positive,DecisionTreeMultiOut,0.587999,6.310839e-09,0.214028,5.350892e-02,0.712429,1.171802,82
126,DAN,positive,RandomForest,0.584580,1.503985e-20,0.639602,1.936323e-25,0.843805,0.794660,209
51,CHCA,positive,SVR_rbf,0.571282,3.850723e-32,0.466665,1.340881e-20,1.396499,0.989684,355
88,ClCCA,negative,RandomForestMultiOut,0.569279,1.123666e-06,0.124220,3.320555e-01,0.381756,1.053334,63
76,CMBT,positive,SVR_poly,0.561497,5.338479e-24,0.491326,6.183796e-18,0.997073,0.894384,272


### Matrix prediction accuracy
Check if the best matrix is selected for each ion with a selected regressor


Ignore molecules that were never observed

In [26]:
# Get rid of molecules that were never detected from results dt:
g = regression_results.groupby(['name_short', 'adduct'], as_index=False)["observed_value"].max()
regression_results = pd.merge(regression_results, g[g["observed_value"] >= 0.][["adduct", "name_short"]],
        how="inner")

accuracy_df = pd.DataFrame(columns = ['regressor', 'accuracy'])
for i, selected_regressor in enumerate(regression_results["regressor"].unique()):
    accuracy = 0
    for (molecule, adduct), rows in regression_results[regression_results.regressor == selected_regressor].groupby(['name_short', 'adduct']):
        # print(rows["observed_value"].idxmax(), rows.loc[rows["observed_value"].idxmax(), ["matrix", "polarity"]])
        best_observed = rows.loc[rows["observed_value"].idxmax(), ["matrix", "polarity"]]
        # Warning: idxmax() returns the first row (even in case of ties)
        best_predicted = rows.loc[rows["prediction"].idxmax()][["matrix", "polarity"]]

        # If all observed intensities were zeros, let's check if all predicted intensities were also low:
        if rows["observed_value"].max() <= 0.:
            # FIXME: for the moment we use arbitrary number
            # print(rows["prediction"].max())
            if rows["prediction"].max() <= 0.05: accuracy += 1
        else:
            if (best_observed == best_predicted).all(): accuracy += 1
        # else:
        #     print("Warning: multiple predictions with same intensity")
        #     if len(pd.merge(best_observed, best_predicted)) > 0: accuracy += 1

    accuracy = accuracy / regression_results[regression_results.regressor == selected_regressor][['name_short', 'adduct']].drop_duplicates().shape[0]
    accuracy_df.loc[i] = [selected_regressor, accuracy]


In [27]:
accuracy_df.sort_values("accuracy", ascending=False)

,regressor,accuracy
5,DecisionTree,0.267963
7,RandomForest,0.240913
4,KNeighbors,0.109890
12,GaussianProcessMultiOut,0.109890
3,SVR_poly,0.081150
11,GaussianProcess,0.076078
2,SVR_rbf,0.065934
10,MLPMultiOut,0.010144
0,Lin_reg,0.005072
1,Lin_regMultiOut,0.005072
